In [2]:
#Import functions
import polars as pl
from itertools import product, combinations
import numpy as np
from numba import cuda

In [3]:
#Load prices and timestamps from CSVs, just AAPL for now, TSLA soon
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)
openPrices=currentDf['open'].to_numpy()
closePrices=currentDf['close'].to_numpy()
timestamps=currentDf['time'].to_numpy()

In [4]:
#Join data into one 2d array
dataArray = np.dstack((openPrices,closePrices, timestamps))[0]

In [11]:
#Algo - takes in parameters and an array with all the data - returns a list of returns
#Numba GPU for speed
@cuda.jit
def algo(allParamsList, dataArray, percents, returns, worked):
    index = cuda.grid(1)
    paramsList=allParamsList[index]
    fee = paramsList[0][0]
    timeout = int(paramsList[0][1])
    allParams=paramsList[1:]

    tradePercent=0
    totalReturn=1

    total=0

    start = 60
    end = int(len(dataArray)-(61))

    if index < allParamsList.size:
        for j in range(start,end):

            #Check that its not within the first or last hour of trading
            if dataArray[j][2]-dataArray[j-60][2]<4500000 and dataArray[j+60][2]-dataArray[j][2]<4500000:
                
                #Set start price and continue
                startPrice=dataArray[j][0]
                continueParams=True

                for param in allParams:
                    ticksBefore = int(param[0])
                    change = param[1]
                    version = param[2]

                    #Set price we are checking
                    if version==1:
                        checkPrice=dataArray[j-ticksBefore][0]

                    elif version==3:
                        #checkPrice=sma(ticksBefore, j, openPrices)
                        checkTotal = 0
                        #Simple mean by add and divide
                        for k in range(j-ticksBefore+1,j+1):
                            checkTotal+=int(dataArray[k][0])
                        checkPrice = checkTotal/ticksBefore
                    
                    #If price now is above price we are checking, then continue
                    if not checkPrice*(change+0.00002)>startPrice>checkPrice*change:
                        continueParams = False

                if continueParams:

                    #Add to tally
                    total+=1
                    endPrice=dataArray[j+timeout][0]

                    #Return on investment = new price / old price
                    returnValue = endPrice/startPrice
                    totalReturn*=(returnValue-fee)
                    tradePercent+=(returnValue-fee)


    if total>0:
        worked[index] = True
        percents[index] = tradePercent/total
        returns[index] = totalReturn

def launchAlgo(allParamsList):
    percents = np.zeros(len(allParamsList), dtype=float)
    returns = np.zeros(len(allParamsList), dtype=float)
    worked = np.zeros(len(allParamsList), dtype=bool)
    threads_per_block = 1024
    blocks_per_grid = (allParamsList.size + (threads_per_block - 1)) // threads_per_block
    algo[blocks_per_grid, threads_per_block](allParamsList, dataArray, percents, returns, worked)
    return (returns[worked], percents[worked], allParamsList[worked])

In [13]:
#Test algo to compile it
testArray = np.array([[[0, 50.00, 0],[10, 1.0, 3]]])
launchAlgo(testArray)

(array([1.02067872]),
 array([1.00039522]),
 array([[[ 0., 50.,  0.],
         [10.,  1.,  3.]]]))

In [16]:
#Params
#rough commission fee
fee=[0.00]

#change
changeList=np.arange(1.000,1.001,0.00002)

#timeout
timeoutList=[1,2,3,5,10,20,30,50]

#sma price before
smaTicksBeforeList=[3,5,10,20,50]
#days before list
priceTicksBeforeList=[1,2,3,5,10,20,30,50]


priceParamProduct = list(product(priceTicksBeforeList, changeList, [1]))
smaParamProduct = list(product(smaTicksBeforeList, changeList, [3]))
allParamProduct = priceParamProduct + smaParamProduct

allSettingsProduct = list(product(fee, timeoutList, fee))

depth = 1
xParamCombinations = list(combinations(allParamProduct, depth))

fullCombinations=[]
for a in allSettingsProduct:
    for b in xParamCombinations:
        fullCombinations.append(([a] + [c for c in b]))
allParamsList = np.array(fullCombinations)
len(allParamsList)

5200

In [18]:
#Run algo
results=launchAlgo(allParamsList)

In [19]:
results[0][0], results[1][0], results[2][0]


(1.0067886268610784,
 1.0000027916045195,
 array([[0., 1., 0.],
        [1., 1., 1.]]))